In this notebook, we will be working with event planning task and also:
- Explore the **Task** parameter in CrewAI

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [6]:
from crewai import Task, Crew, Agent, LLM

In [3]:
import os
from dotenv import load_dotenv
load_dotenv()



True

In [4]:
os.environ['GROQ_API_KEY']= os.getenv("GROQ_API_KEY")
os.environ['SERPER_API_KEY'] = os.getenv("SERPER_API_KEY")

In [68]:
llm = LLM(model="groq/llama-3.1-8b-instant")

## Let's create Tools

In [69]:
from crewai_tools import SerperDevTool, ScrapeWebsiteTool

In [70]:
# Initialize the tools
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Let's create Agents

In [71]:
# Agent 1: Venue Coordinator
venue_coordinator = Agent(
    role="Venue Coordinator",
    goal="Identify and book an appropriate venue "
    "based on event requirements",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "With a keen sense of space and "
        "understanding of event logistics, "
        "you excel at finding and securing "
        "the perfect venue that fits the event's theme, "
        "size, and budget constraints."
    ),
    llm=llm
)

In [72]:
 # Agent 2: Logistics Manager
logistics_manager = Agent(
    role='Logistics Manager',
    goal=(
        "Manage all logistics for the event "
        "including catering and equipmen"
    ),
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Organized and detail-oriented, "
        "you ensure that every logistical aspect of the event "
        "from catering to equipment setup "
        "is flawlessly executed to create a seamless experience."
    ),
    llm=llm
)

In [73]:
# Agent 3: Marketing and Communications Agent
marketing_communications_agent = Agent(
    role="Marketing and Communications Agent",
    goal="Effectively market the event and "
         "communicate with participants",
    tools=[search_tool, scrape_tool],
    verbose=True,
    backstory=(
        "Creative and communicative, "
        "you craft compelling messages and "
        "engage with potential attendees "
        "to maximize event exposure and participation."
    ),
    llm=llm
)

## Let's Create Pydantic Models for the Event

In [74]:
from pydantic import BaseModel

class VenueDetails(BaseModel):
    name: str
    address: str
    capacity: int
    booking_status: str

## Let's create Tasks

Now we will play with different parameters of the agent.
- async_execution=True mean that the task can be executed with the task coming after it parallely.

In [75]:
venue_task = Task(
    description="Find a venue in {event_city} "
                "that meets criteria for {event_topic}.",
    expected_output="All the details of a specifically chosen"
                    "venue you found to accommodate the event.",
    human_input=True,
    output_json=VenueDetails,
    output_file="venue_details.json",  
      # Outputs the venue details as a JSON file
    agent=venue_coordinator,
    # llm=llm
)

In [76]:
logistics_task = Task(
    description="Coordinate catering and "
                 "equipment for an event "
                 "with {expected_participants} participants "
                 "on {tentative_date}.",
    expected_output="Confirmation of all logistics arrangements "
                    "including catering and equipment setup.",
    human_input=True,
    async_execution=True,
    agent=logistics_manager,
    # llm=llm
)

In [77]:
marketing_task = Task(
    description="Promote the {event_topic} "
                "aiming to engage at least"
                "{expected_participants} potential attendees.",
    expected_output="Report on marketing activities "
                    "and attendee engagement formatted as markdown.",
    # async_execution=True,
    output_file="marketing_report.md",  # Outputs the report as a text file
    agent=marketing_communications_agent,
    # llm=llm
)

## Let's create a crew

In [78]:
# Define the crew with agents and tasks
event_management_crew = Crew(
    agents=[venue_coordinator, 
            logistics_manager, 
            marketing_communications_agent],
    
    tasks=[venue_task, 
           logistics_task, 
           marketing_task],
    
    verbose=True
)

In [79]:
event_details = {
    'event_topic': "Marriage Ceremony",
    'event_description': "A gathering of top-notch "
                         "people to attend the event"
                         "of business tycoon son wedding.",
    'event_city': "Lahore",
    'tentative_date': "2025-04-19",
    'expected_participants': 500,
    'budget': 20000,
    'venue_type': "Marriage Hall"
}

In [80]:
result = event_management_crew.kickoff(inputs=event_details)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 503625aa-3e56-4109-a051-bbe828eb1fa8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

🚀 Crew: crew
└── 📋 Task: 17b53b2f-0127-483f-a935-50abd70482c8
       Status: Executing Task...

🚀 Crew: crew
└── 📋 Task: 17b53b2f-0127-483f-a935-50abd70482c8
       Status: Executing Task...
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress

# Agent: Venue Coordinator
## Task: Find a venue in Lahore that meets criteria for Marriage Ceremony.


🤖 Agent: Venue Coordinator
    Status: In Progress

...

 Error during LLM call: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jntxkgfbejvarxk9dccvc5b7` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4371, Requested 3793. Please try again in 21.631s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}

 An unknown error occurred. Please check the details below.
 Error details: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jntxkgfbejvarxk9dccvc5b7` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4371, Requested 3793. Please try again in 21.631s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}



🚀 Crew: crew
└── 📋 Task: 17b53b2f-0127-483f-a935-50abd70482c8
       Assigned to: Venue Coordinator
       Status: ❌ Failed
    ├── 🤖 Agent: Venue Coordinator
    │       Status: In Progress
    └── 🤖 Agent: Venue Coordinator
            Status: In Progress
        └── ❌ LLM Failed

╭───────────────────────────────────────────────── Task Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Failed                                                                                                    │
│  Name: 17b53b2f-0127-483f-a935-50abd70482c8                                                                     │
│  Agent: Venue Coordinator                                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────── Crew Failure ──────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Failed                                                                                          │
│  Name: crew                                                                                                     │
│  ID: 503625aa-3e56-4109-a051-bbe828eb1fa8                                                                       │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

RateLimitError: litellm.RateLimitError: RateLimitError: GroqException - {"error":{"message":"Rate limit reached for model `llama-3.1-8b-instant` in organization `org_01jntxkgfbejvarxk9dccvc5b7` service tier `on_demand` on tokens per minute (TPM): Limit 6000, Used 4371, Requested 3793. Please try again in 21.631s. Need more tokens? Upgrade to Dev Tier today at https://console.groq.com/settings/billing","type":"tokens","code":"rate_limit_exceeded"}}
